In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
from sklearn import metrics

In [ ]:
workdir=os.getcwd()
os.makedir('fig')
fig_dir=workdir+'/fig'
algo_name=['T23', 'S', 'S0', 'S10', 'S20', 'T', 'T0', 'T10', 'T20']
algo={}
for i in range(len(algo_name)):
    algo[i]=pd.read_csv(workdir+'/data/{}.csv'.format(algo_name[i]))

In [ ]:
%%time
#cut
ptcut=10
etamin=1.6
etamax=2.9
algo_cut={}
for i in algo:
    sel=algo[i]['genpart_pt']>ptcut
    algo_cut[i]=algo[i][sel]
    sel=np.abs(algo_cut[i]['genpart_exeta'])>etamin
    algo_cut[i]=algo_cut[i][sel]
    sel=np.abs(algo_cut[i]['genpart_exeta'])<etamax
    algo_cut[i]=algo_cut[i][sel]
    algo_cut[i].dropna(inplace=True)
    algo_cut[i]['genpart_pid'].replace([-11,11],0, inplace=True)
    algo_cut[i]['genpart_pid'].replace([-211,211],1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
columns=['cl3d_eta','cl3d_showerlength',
       'cl3d_coreshowerlength', 'cl3d_firstlayer', 'cl3d_maxlayer', 'cl3d_szz',
       'cl3d_seetot', 'cl3d_spptot', 'cl3d_srrtot', 'cl3d_srrmean','cl3d_pt']


X_train={}
X_test={}
y_train={}
y_test={}

for i in algo:
    X_train[i], X_test[i], y_train[i], y_test[i] = train_test_split(algo_cut[i][columns], algo_cut[i]['genpart_pid'], test_size=0.2)

In [ ]:
X_pt={}
for i in algo:
    X_pt[i]=X_test[i]['cl3d_pt']
    X_test[i]=X_test[i].drop(columns='cl3d_pt')
    X_train[i]=X_train[i].drop(columns='cl3d_pt')
    
columns.remove('cl3d_pt')


In [ ]:
train={}
test={}

for i in algo:
    train[i] = xgb.DMatrix(data=X_train[i],label=y_train[i], feature_names=columns)
    test[i] = xgb.DMatrix(data=X_test[i],label=y_test[i],feature_names=columns)

In [ ]:
def plot_discr(feature):
    nbins=20
    plt.hist(X_train[i][feature][y_train[i] == 0],
         histtype='step',color='midnightblue',label='electrons', bins=nbins);
    plt.hist(X_train[i][feature][y_train[i] == 1],
         histtype='step',color='firebrick',label='pions', bins=nbins);
    plt.xlabel(feature,fontsize=12);
    plt.ylabel('Events',fontsize=12);
    plt.legend(frameon=False);

In [ ]:
%%time
for i in algo:
    plt.figure(figsize=(15,30))
    j=0
    for feature in columns:
        j+=1
        plt.subplot(4,3,j)
        plot_discr(feature)
        plt.title(algo_name[i])


In [ ]:
%%time
#corr matrices for signal/background
os.makedirs(name='corr_matrix', exist_ok=True)
corrsig={}
corrbac={}
for i in algo:
    sel1=algo_cut[i]['genpart_pid']==0
    sel2=algo_cut[i]['genpart_pid']==1
    corrsig[i]=algo_cut[i][sel1][columns].corr()
    corrbac[i]=algo_cut[i][sel2][columns].corr()
    fig, ax = plt.subplots(ncols=2,figsize=(20, 10), sharey=True, sharex=True)
    fig.suptitle(algo_name[i])
    
    axs=ax[0]
    cax=axs.matshow(corrsig[i], cmap='PiYG', vmin=-1, vmax=1)
    plt.xticks(range(len(corrsig[i].columns)), corrsig[i].columns);
    plt.yticks(range(len(corrsig[i].columns)), corrsig[i].columns);
    axs.set_title('signal', pad=70)
    
    plt.colorbar(cax,ax=axs)
    for item in (axs.get_xticklabels()):
        item.set_rotation(90)
    
    axs=ax[1]
    cax=axs.matshow(corrbac[i], cmap='PiYG', vmin=-1,vmax=1)
    plt.xticks(range(len(corrbac[i].columns)), corrbac[i].columns);
    plt.yticks(range(len(corrbac[i].columns)), corrbac[i].columns);
    axs.set_title('background', pad=70)
    
    plt.colorbar(cax,ax=axs)
    
    for item in (axs.get_xticklabels()):
        item.set_rotation(90)
    
    plt.savefig(fig_dir+'corr_matrix/correlation_matrix_%s.png' %algo_name[i])

In [ ]:
%%time
#violin plots for 1 algo
i=1
algo_cut[1]['all']=''
algo_cut[i]['electron']=(algo_cut[i]['genpart_pid']==0)
j=0
plt.figure(figsize=(15,25))

for  feature in columns:
    j+=1
    plt.subplot(4,3,j)
    ax=sns.violinplot(y=feature, x='all',hue='electron', data=algo_cut[1], split=True)
    ax.set_xlabel("")
    
plt.suptitle(algo_name[i])
plt.savefig(fig_dir+'violinplot.png')

In [ ]:
%%time
for i in algo:        
    algo_cut[i]['algo']=algo_name[i]
    algo_cut[i]['electron']=(algo_cut[i]['genpart_pid']==0)
    if i==0:
        algo_all=algo_cut[i]
    else:
        algo_all=pd.concat([algo_all,algo_cut[i]])


In [ ]:
%%time
#violin plot for all algos

j=0
plt.figure(figsize=(25,60))

for  feature in columns:
    j+=1
    plt.subplot(10,1,j)
    ax=sns.violinplot(y=feature, x='algo',hue='electron', data=algo_all, split=True)
    if j==4:
        ax.set_ylim(top=18)
    if j==5:
        ax.set_ylim(8,35)
    if j==6:
        ax.set_ylim(top=30)   
    
plt.suptitle('Violin plot')
plt.savefig(fig_dir'violinplotall.png')

In [ ]:
param = {}

# Booster parameters
param['nthread']          = 30  # limit number of threads
param['eta']              = 0.1 # learning rate
param['max_depth']        = 10  # maximum depth of a tree
param['subsample']        = 0.8 # fraction of events to train tree on
param['colsample_bytree'] = 0.8 # fraction of features to train tree on
param['silent'] = True

# Learning task parameters
param['objective']   = 'binary:logistic' # objective function
param['eval_metric'] = 'error'           # evaluation metric for cross validation
param = list(param.items()) + [('eval_metric', 'logloss')] + [('eval_metric', 'rmse')]

num_trees = 100  # number of trees to make

In [ ]:
%%time
booster={}
for i in algo:
    booster[i] = xgb.train(param,train[i],num_boost_round=num_trees);

In [ ]:
predictions={}
for i in algo:
    print(algo_name[i],'\t',booster[i].eval(test[i]))
    predictions[i]=booster[i].predict(test[i])

In [ ]:
for i in algo:
# plot all predictions (both signal and background)
    plt.figure(figsize=(20,5));

    plt.subplot(121);
    plt.hist(predictions[i],bins=np.linspace(0,1,50),histtype='step',color='darkgreen',label='All events');
# make the plot readable
    plt.xlabel('Prediction from BDT',fontsize=12);
    plt.ylabel('Events',fontsize=12);
    plt.title(algo_name[i])
    plt.legend(frameon=False);

# plot signal and background separately
    plt.subplot(122);
    plt.hist(predictions[i][test[i].get_label().astype(bool)],bins=np.linspace(0,1,50),
         histtype='step',color='midnightblue',label='signal');
    plt.hist(predictions[i][~(test[i].get_label().astype(bool))],bins=np.linspace(0,1,50),
         histtype='step',color='firebrick',label='background');
# make the plot readable
    plt.xlabel('Prediction from BDT',fontsize=12);
    plt.ylabel('Events',fontsize=12);
    plt.title(algo_name[i])
    plt.legend(frameon=False);


In [ ]:
plt.figure(figsize=(20,25));
for i in algo:
# plot all predictions (both signal and background)
    
    plt.subplot(4,3,i+1)
# plot signal and background separately
    plt.hist(predictions[i][test[i].get_label().astype(bool)],bins=np.linspace(0,1,50),
         histtype='step',color='midnightblue',label='signal');
    plt.hist(predictions[i][~(test[i].get_label().astype(bool))],bins=np.linspace(0,1,50),
         histtype='step',color='firebrick',label='background');
# make the plot readable
    plt.xlabel('Prediction from BDT',fontsize=12);
    plt.ylabel('Events',fontsize=12);
    plt.title(algo_name[i])
    plt.legend(frameon=False);
plt.savefig(fig_dir+'predictions.png')

In [ ]:
#plot importance of features for each algo
plt.figure(figsize=(20,30))
for i in algo:
    ax=plt.subplot(5,2,i+1)
    xgb.plot_importance(booster[i],ax,grid=False, title=algo_name[i], importance_type='gain');
plt.savefig(fig_dir'importance.png')

In [ ]:
#build ROC

plt.figure(figsize=(10,5))
for i in algo:
    #buildROC(y_test[i], predictions[i])
    fpr, tpr, threshold = metrics.roc_curve(y_test[i],predictions[i])
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(tpr,(1-fpr), label ='%s AUC = %0.4f' %(algo_name[i],roc_auc))
plt.legend(loc = 'lower left')
    #plt.plot([0, 1], [0, 1],'r--')
plt.xlim(0.95,1)
plt.yscale('log')
    #plt.ylim(0.6,1.05)
plt.xlabel('efficiency')
plt.ylabel('Log Rejection Rate') 
plt.savefig(fig_dir+'ROC.png')

In [ ]:
def score(ytest, ypred, thr):
    fpr, tpr, threshold = metrics.roc_curve(ytest,ypred)
    roc=pd.DataFrame({'tpr':tpr,'fpr':fpr, 'threshold':threshold})
    roc_cut=roc[roc['tpr']>thr];
    score=np.min(roc_cut['fpr']);
    return score

In [ ]:
thr=0.995
scor=[]
for i in algo:
    scor.append(score(y_test[i], predictions[i],thr))
    print('rejection rate for %s at %0.3f threshold=' %(algo_name[i],thr) ,scor[i])

In [ ]:
#plot background efficiency
plt.figure(figsize=(10,5))
plt.bar(np.arange(len(algo_name)), scor)
plt.xticks(np.arange(len(algo_name)), algo_name);
plt.ylabel('Background efficiency')
plt.savefig(fig_dir+'efficiency.png')

In [ ]:
#CALCULATE ERROR BARS
conf_level=0.99
from scipy import stats
#normal
#def error(total, score, conf_level):
#    alpha=(1-conf_level)/2
#    sigma=np.sqrt(score*(1-score)/total)
#    delta=np.abs(score-stats.norm.ppf(1-alpha,loc=score, scale=sigma))
#    return delta

#clopper pearson
def error(total, score, conf_level):
    alpha=(1-conf_level)/2
    n=total
    k=score*n
    lo = score-stats.beta.ppf(alpha/2, k, n-k+1)
    hi = stats.beta.ppf(1 - alpha/2, k+1, n-k)-score
    return lo, hi

In [ ]:
#comparison with old 2d3d algo
thr=0.9
#make the comparison
b=algo_cut[0].dropna()
a=b['cl3d_bdteg']
c=np.interp(a, (a.min(), a.max()), (1, 0))

score_old=score(b['genpart_pid'],c,thr)
print('old bdt score:', score_old)

i=0
nbins=20
plt.figure(figsize=(10,5))
plt.hist(c[b['genpart_pid']==0],histtype='step', bins=nbins, label='electron');
plt.hist(c[b['genpart_pid']==1],histtype='step',  bins=nbins, label='pion');
plt.legend()
i=0

fpr, tpr, threshold = metrics.roc_curve(b['genpart_pid'],c)
roc_auc = metrics.auc(fpr, tpr)
plt.plot(tpr,np.log(1-fpr), label ='old bdteg AUC = %0.4f' %(roc_auc))

fpr, tpr, threshold = metrics.roc_curve(y_test[i],predictions[i])
roc_auc = metrics.auc(fpr, tpr)
plt.plot(tpr,np.log(1-fpr), label ='%s AUC = %0.4f' %(algo_name[i],roc_auc))
    
plt.legend(loc = 'lower left')
    #plt.plot([0, 1], [0, 1],'r--')
plt.xlim(0.8,1)
    #plt.ylim(0.6,1.05)
plt.xlabel('efficiency')
plt.ylabel('Log Rejection Rate')  
plt.savefig(fig_dir+'old.png')

In [ ]:
%%time
#plot score
%matplotlib inline
import matplotlib.ticker as ticker
fig=plt.figure(figsize=(15,20))
thr=0.95
nbins=10
conf_level=0.99
pred_pt={}
y_test_pt={}
bins_pt={}
score_pt={}
y_err_pt={}
#faire les bins après entrainement du bdt
#--> predictions[i] vs y_test[i]
#predictions[i]=booster[i].predict(test[i])--> binner test[i]
# --> binner X_test[i] et y_test[i]
for i in algo:
    
    pt_max=np.max(X_pt[i])
    pt_min=np.min(X_pt[i])
    range_pt=pt_max-pt_min
    bins_pt=[]
    score_pt=[]
    y_err_pt=[]
    y_err_pt_lo=[]
    y_err_pt_hi=[]
    pred_pt={}
    y_test_pt={}
    
    
    X_test[i]['abseta']=np.abs(X_test[i]['cl3d_eta'])
    eta_max=np.max(X_test[i]['abseta'])
    eta_min=np.min(X_test[i]['abseta'])
    range_eta=eta_max-eta_min
    bins_eta=[]
    score_eta=[]
    y_err_eta=[]
    y_err_eta_lo=[]
    y_err_eta_hi=[]
    pred_eta={}
    y_test_eta={}
    for j in range(nbins):
        #make sure last bins are not too small
        n_min=X_pt[i].shape[0]/(1.4*nbins)
        
        sel=(X_pt[i] >= (pt_min+j*range_pt/nbins)) & (X_pt[i] < (pt_min+(j+1)*range_pt/nbins))
        
        if X_pt[i][sel].shape[0]>n_min:
            bins_pt.append(((pt_min+j*range_pt/nbins)+(pt_min+(j+1)*range_pt/nbins))/2)
           
            pred_pt[j]=predictions[i][sel]
            
            y_test_pt[j]=y_test[i][sel]
        else: 
            
            bins_pt.append(((pt_min+j*range_pt/nbins)+(pt_max))/2)
            sel=(X_pt[i] >= (pt_min+j*range_pt/nbins))
            pred_pt[j]=predictions[i][sel]
            y_test_pt[j]=y_test[i][sel]
            
            break       

    for j in range(len(pred_pt)): 
        
        score_pt.append(score(y_test_pt[j], pred_pt[j],thr))
        #print('rejection rate for pt bin %d at %0.3f threshold=' %(j+1,thr),  score_pt[j])
        
        lo,hi=error(y_test_pt[j].shape[0], score(y_test_pt[j], pred_pt[j], thr), conf_level)
        
        y_err_pt_lo.append(lo)
        y_err_pt_hi.append(hi)
        
    
    for j in range(nbins):
        
        bins_eta.append(((eta_min+j*range_eta/nbins)+(eta_min+(j+1)*range_eta/nbins))/2)
        sel=(X_test[i]['abseta'] > (eta_min+j*range_eta/nbins)) & (X_test[i]['abseta'] < (eta_min+(j+1)*range_eta/nbins))
        
        pred_eta[j]=predictions[i][sel]
        
        y_test_eta[j]=y_test[i][sel]
        score_eta.append(score(y_test_eta[j], pred_eta[j],thr))
        lo, hi=error(y_test_eta[j].shape[0], score(y_test_eta[j], pred_eta[j], thr), conf_level)
        y_err_eta_lo.append(lo)
        y_err_eta_hi.append(hi)
        
        
        
    y_err_pt=[y_err_pt_lo, y_err_pt_hi]
    y_err_eta=[y_err_eta_lo, y_err_eta_hi]
        
    plt.subplot(411);
    plt.errorbar(bins_pt, score_pt, y_err_pt, label=algo_name[i]);
    plt.ylabel('Background efficiency score');
    plt.xlabel('pt');
    plt.ylim(-0.001,0.01)
    plt.xlim(right=130)
    plt.legend()
    
    plt.subplot(412);
    plt.errorbar(bins_eta, score_eta, y_err_eta, label=algo_name[i]);
    plt.ylabel('Background efficiency score');
    plt.xlabel('eta');
    plt.ylim(-0.001,0.006)
    plt.legend()
    plt.xlim(right=3.0)
    
    plt.subplot(413);
    plt.errorbar(bins_pt, score_pt, y_err_pt, label=algo_name[i]);
    plt.ylabel('Logit Background efficiency score');
    plt.xlabel('pt');
    plt.xlim(right=130)
    plt.yscale('logit')
    plt.gca().yaxis.set_minor_locator(ticker.LogLocator(subs=[0]));
    plt.ylim(0.000008,0.03)
    plt.legend()
    
    plt.subplot(414);
    plt.errorbar(bins_eta, score_eta, y_err_eta, label=algo_name[i]);
    plt.ylabel('Logit Background efficiency score');
    plt.xlabel('eta');
    plt.yscale('logit')
    plt.gca().yaxis.set_minor_locator(ticker.LogLocator(subs=[0]));
    plt.ylim(0.000008,0.03)
    plt.xlim(right=3.0)
    plt.legend()
    
   
    
    plt.suptitle('Background efficiency at %0.3f signal efficiency at %0.2f CL for different algorithms' %(thr, conf_level), fontsize=14)
    plt.savefig(fig_dir+'Algo_comparison.png')